<a href="https://colab.research.google.com/github/HwangJae-won/Data_project_practice/blob/main/0928_Heart_Disease_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0928 Heart Disease Prediction
[피드백]
- 실제 시험에서는 코랩 환경에서 진행하자. 노트북 로컬에서 진행하니 커널 다운이 계속 됐다
- SVM, Ridge, Lasso 등 단순 모델도 성능 확인해볼걸 그랬다
- 모델링 시에 쓰는 파라미터 종류들 좀 더 공부해둘 것
-
---


## 데이터 전처리

In [ ]:
#전처리
import pandas as pd
import numpy as np
#시각화
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv("/Users/hwangjaewon/Desktop/Heart_Disease_Prediction.csv")

In [ ]:
df.head()

,Age,Sex,Chest pain type,BP,Cholesterol,FBS over 120,EKG results,Max HR,Exercise angina,ST depression,Slope of ST,Number of vessels fluro,Thallium,Heart Disease
0,70,1,4,130,322,0,2,109,0,2.4,2,3,3,Presence
1,67,0,3,115,564,0,2,160,0,1.6,2,0,7,Absence
2,57,1,2,124,261,0,0,141,0,0.3,1,0,7,Presence
3,64,1,4,128,263,0,0,105,1,0.2,2,1,7,Absence
4,74,0,2,120,269,0,2,121,1,0.2,1,1,3,Absence


target 변수 0,1로 변환해주기

In [ ]:
df= df.replace({'Heart Disease': {'Presence': 1, 'Absence': 0}})

In [ ]:
df.info()
#결측치 없음

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 270 entries, 0 to 269
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Age                      270 non-null    int64  
 1   Sex                      270 non-null    int64  
 2   Chest pain type          270 non-null    int64  
 3   BP                       270 non-null    int64  
 4   Cholesterol              270 non-null    int64  
 5   FBS over 120             270 non-null    int64  
 6   EKG results              270 non-null    int64  
 7   Max HR                   270 non-null    int64  
 8   Exercise angina          270 non-null    int64  
 9   ST depression            270 non-null    float64
 10  Slope of ST              270 non-null    int64  
 11  Number of vessels fluro  270 non-null    int64  
 12  Thallium                 270 non-null    int64  
 13  Heart Disease            270 non-null    int64  
dtypes: float64(1), int64(13)
m

target 변수의 비율 확인

In [ ]:
df['Heart Disease'].value_counts()/len(df)
#불균형은 보이지 않음

0    0.555556
1    0.444444
Name: Heart Disease, dtype: float64

In [ ]:
# sns.pairplot(df)
#모든 변수에 대한 pairplot: 시간 너무 오래걸림 얘는 빼자

In [ ]:
X= df.drop(["Heart Disease"],axis=1)
y= df[["Heart Disease"]]

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [ ]:
# 정규화, 인코딩
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 270 entries, 0 to 269
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Age                      270 non-null    int64  
 1   Sex                      270 non-null    int64  
 2   Chest pain type          270 non-null    int64  
 3   BP                       270 non-null    int64  
 4   Cholesterol              270 non-null    int64  
 5   FBS over 120             270 non-null    int64  
 6   EKG results              270 non-null    int64  
 7   Max HR                   270 non-null    int64  
 8   Exercise angina          270 non-null    int64  
 9   ST depression            270 non-null    float64
 10  Slope of ST              270 non-null    int64  
 11  Number of vessels fluro  270 non-null    int64  
 12  Thallium                 270 non-null    int64  
dtypes: float64(1), int64(12)
memory usage: 27.5 KB


## 수치형과 범주형 변수 나누어 인코딩, 스케일링 진행

In [ ]:
numerical_ix = X.columns[[0, 3, 4, 5] ]
categorical_ix = X.columns[[1, 2, 5, 6, 8,10,11,12 ]]

In [ ]:
t = [('cat', OneHotEncoder(handle_unknown = "ignore"), categorical_ix), ('num', StandardScaler(), numerical_ix)]
col_transform = ColumnTransformer(transformers=t)

In [ ]:
X_train_std = col_transform.fit_transform(X_train)
X_test_std = col_transform.transform(X_test)

In [ ]:
X_train_std

array([[ 0.        ,  1.        ,  1.        , ..., -0.67478847,
        -1.05780979, -0.41763453],
       [ 1.        ,  0.        ,  0.        , ...,  1.67028286,
        -0.90567196, -0.41763453],
       [ 0.        ,  1.        ,  0.        , ..., -1.84732413,
        -0.50631017, -0.41763453],
       ...,
       [ 1.        ,  0.        ,  0.        , ...,  4.01535418,
         0.7488269 ,  2.394438  ],
       [ 0.        ,  1.        ,  0.        , ..., -1.2610563 ,
        -0.98174087, -0.41763453],
       [ 0.        ,  1.        ,  0.        , ..., -1.84732413,
         0.95801641, -0.41763453]])

## Modlening and Tuning

In [ ]:
#classifier 설정
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
#튜닝 방법:grid
from sklearn.model_selection import GridSearchCV

In [ ]:
#model 정의
models = {
    # 'RandomForest': (RandomForestClassifier(),
    #              {
    #                  'n_estimators': [50, 100],
    #                  'max_depth': [2, 20],
    #                  'min_samples_split':[5, 100],
    #              }),
    # 'LightGBM': (LGBMClassifier(),
    #             {
    #                  'n_estimators': [50, 100],
    #                  'max_depth': [2, 20],
    #                  'min_child_samples':[5, 100],
    #             }),
    #LGBM 계속 커널 오류
    'GB': (GradientBoostingClassifier(),
                 {
                     'n_estimators': [50, 100],
                     'max_depth': [2, 20],
                     'min_samples_split':[5, 100],
                 }),
    'ExtraTree': (ExtraTreesClassifier(),
                  {
                     'n_estimators': [50, 100],
                     'max_depth': [2, 20],
                     'min_samples_split':[5, 100],
                 }),

    'Catboost': (CatBoostClassifier(),
                 {
                    'depth': [4, 5, 6],
                    'leaf_estimation_iterations': [10]
                 })
}

In [ ]:
models.items()

dict_items([('GB', (GradientBoostingClassifier(), {'n_estimators': [50, 100], 'max_depth': [2, 20], 'min_samples_split': [5, 100]})), ('ExtraTree', (ExtraTreesClassifier(), {'n_estimators': [50, 100], 'max_depth': [2, 20], 'min_samples_split': [5, 100]})), ('Catboost', (<catboost.core.CatBoostClassifier object at 0x7f7fa262e1c0>, {'depth': [4, 5, 6], 'leaf_estimation_iterations': [10]}))])

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate

In [ ]:
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [ ]:
model_results2 = []

# 모델별로 튜닝 및 적합
for model_name, (model, param_grid) in models.items():
    print(f"Start {model_name} Training!")
    search = GridSearchCV(model, param_grid, scoring='accuracy',  n_jobs=-1)
    search.fit(X_train_std, y_train)

    # 최적의 모델과 하이퍼파라미터 출력
    best_model =search.best_estimator_
    best_params =search.best_params_
    print(f"Best Parameters: {best_params}")

    # 테스트 데이터에 대한 예측 및 평가
    y_pred = best_model.predict(X_test_std)
    f1 = f1_score(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)
    c_m = confusion_matrix(y_test, y_pred)
    print(f"F1 score: {f1}\n")
    print(f"Accuracy: {accuracy}\n")
    print(f"Confusion Matrix: {c_m}\n")


    # 모델 이름과 accuracy 결과 저장
    model_results2.append((model_name, accuracy, f1))

Start GB Training!


/Users/hwangjaewon/anaconda3/lib/python3.8/site-packages/sklearn/ensemble/_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/hwangjaewon/anaconda3/lib/python3.8/site-packages/sklearn/ensemble/_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/hwangjaewon/anaconda3/lib/python3.8/site-packages/sklearn/ensemble/_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/hwangjaewon/anaconda3/lib/python3.8/site-packages/sklearn/ensemble/_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please 

Best Parameters: {'max_depth': 2, 'min_samples_split': 100, 'n_estimators': 50}
F1 score: 0.7999999999999999

Accuracy: 0.8235294117647058

Confusion Matrix: [[32  8]
 [ 4 24]]

Start ExtraTree Training!


/Users/hwangjaewon/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/hwangjaewon/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/hwangjaewon/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/hwangjaewon/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_valida

Best Parameters: {'max_depth': 2, 'min_samples_split': 5, 'n_estimators': 50}
F1 score: 0.7407407407407408

Accuracy: 0.7941176470588235

Confusion Matrix: [[34  6]
 [ 8 20]]

Start Catboost Training!
0:	learn: 0.6669597	total: 68.8ms	remaining: 1m 8s
0:	learn: 0.6647851	total: 68.7ms	remaining: 1m 8s
0:	learn: 0.6717684	total: 68.1ms	remaining: 1m 8s
0:	learn: 0.6726696	total: 67.9ms	remaining: 1m 7s
0:	learn: 0.6741059	total: 67.8ms	remaining: 1m 7s
0:	learn: 0.6675053	total: 67.9ms	remaining: 1m 7s
0:	learn: 0.6739663	total: 68ms	remaining: 1m 7s
0:	learn: 0.6647750	total: 68.3ms	remaining: 1m 8s
1:	learn: 0.6495460	total: 72ms	remaining: 35.9s
1:	learn: 0.6587598	total: 71.9ms	remaining: 35.9s
1:	learn: 0.6521046	total: 72.4ms	remaining: 36.1s
1:	learn: 0.6459569	total: 74.7ms	remaining: 37.3s
2:	learn: 0.6435882	total: 73.8ms	remaining: 24.5s
1:	learn: 0.6430136	total: 74.6ms	remaining: 37.2s
2:	learn: 0.6443386	total: 74.8ms	remaining: 24.9s
2:	learn: 0.6356993	total: 74.4ms	rema

In [ ]:
model_results2

[('GB', 0.8235294117647058, 0.7999999999999999),
 ('ExtraTree', 0.7941176470588235, 0.7407407407407408),
 ('Catboost', 0.7647058823529411, 0.7142857142857143)]

In [ ]:
model_results

[('RandomForest', 0.7941176470588235),
 ('GB', 0.8235294117647058),
 ('Catboost', 0.7647058823529411)]

GB, ExtraTree, RF가 높음

In [ ]:
models

{'RandomForest': (RandomForestClassifier(),
  {'n_estimators': [50, 100],
   'max_depth': [2, 20],
   'min_samples_split': [5, 100]}),
 'GB': (GradientBoostingClassifier(),
  {'n_estimators': [50, 100],
   'max_depth': [2, 20],
   'min_samples_split': [5, 100]}),
 'Catboost': (<catboost.core.CatBoostClassifier at 0x7fe0e358bee0>,
  {'depth': [4, 5, 6], 'leaf_estimation_iterations': [10]})}

In [ ]:
models

{'GB': (GradientBoostingClassifier(),
  {'n_estimators': [50, 100],
   'max_depth': [2, 20],
   'min_samples_split': [5, 100]}),
 'ExtraTree': (ExtraTreesClassifier(),
  {'n_estimators': [50, 100],
   'max_depth': [2, 20],
   'min_samples_split': [5, 100]}),
 'Catboost': (<catboost.core.CatBoostClassifier at 0x7f7fa262e1c0>,
  {'depth': [4, 5, 6], 'leaf_estimation_iterations': [10]})}

In [ ]:
from sklearn.ensemble import VotingClassifier

# VotingClassifier
vc = VotingClassifier(models, voting='soft')
voting_classifier = VotingClassifier(estimators=models)

In [ ]:
voting_classifier.fit(X_train_std, y_train)

In [ ]:
y_pred = voting_classifier.predict(X_test_std)
f1 = f1_score(y_test, y_pred)
c_m = confusion_matrix(y_test, y_pred)
print(f"F1 score: {f1}\n")
print(f"Accuracy: {accuracy}\n")
print(f"Confusion Matrix: {c_m}\n")

F1 score: 0.736842105263158

Accuracy: 0.7647058823529411

Confusion Matrix: [[32  8]
 [ 7 21]]



In [ ]:
# hard VotingClassifier
vc = VotingClassifier(models, voting='hard')
voting_classifier = VotingClassifier(estimators=class_models)
voting_classifier.fit(X_train_std, y_train)
y_pred = voting_classifier.predict(X_test_std)
f1 = f1_score(y_test, y_pred)
c_m = confusion_matrix(y_test, y_pred)
print(f"F1 score: {f1}\n")
print(f"Accuracy: {accuracy}\n")
print(f"Confusion Matrix: {c_m}\n")

/Users/hwangjaewon/anaconda3/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/hwangjaewon/anaconda3/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Learning rate set to 0.005204
0:	learn: 0.6885518	total: 6.64ms	remaining: 6.63s
1:	learn: 0.6849154	total: 9.14ms	remaining: 4.56s
2:	learn: 0.6802432	total: 11ms	remaining: 3.65s
3:	learn: 0.6764904	total: 36.5ms	remaining: 9.08s
4:	learn: 0.6728923	total: 38.4ms	remaining: 7.65s
5:	learn: 0.6686715	total: 40.8ms	remaining: 6.76s
6:	learn: 0.6646237	total: 41.6ms	remaining: 5.89s
7:	learn: 0.6611511	total: 43.2ms	remaining: 5.36s
8:	learn: 0.6574098	total: 52.2ms	remaining: 5.74s
9:	learn: 0.6540732	total: 53.3ms	remaining: 5.27s
10:	learn: 0.6502840	total: 54.9ms	remaining: 4.94s
11:	learn: 0.6464483	total: 57.2ms	remaining: 4.71s
12:	learn: 0.6419893	total: 59.7ms	remaining: 4.54s
13:	learn: 0.6383739	total: 71.7ms	remaining: 5.05s
14:	learn: 0.6346390	total: 74.3ms	remaining: 4.88s
15:	learn: 0.6314707	total: 76.9ms	remaining: 4.73s
16:	learn: 0.6275200	total: 80.1ms	remaining: 4.63s
17:	learn: 0.6234357	total: 97.2ms	remaining: 5.3s
18:	learn: 0.6196385	total: 100ms	remaining: 5.

In [ ]:
#soft voting 결과
F1 score: 0.736842105263158

Accuracy: 0.7647058823529411

#### accuracy
- RandomForest, 0.7941176470588235
- GB, **0.823529**4117647058
- Catboost, 0.7647058823529411
- ExtraTree', 0.7941176470588235
- sofr voting: 0.764

----
**GB 단독이 성능 더 높음:  0.8235**
